In [ ]:
is_it_a_test = False
test_name = "_feature"

In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

In [ ]:
# prepare this session

In [ ]:
! python -m luigi --module src.luigi_pipeline PrepareNextSession --local-scheduler

In [ ]:
import numpy as np
import pandas as pd

from src.data.collect_data import *
import src.data.get_dataset as get_dataset

In [ ]:
vocab = initiate_vocab(test=is_it_a_test, test_name=test_name)

all_vocab = get_dataset.get_all_vocab()

historical_data, tries_session, nb_known_words_session = initiate_session()

In [ ]:
for i in range(100):

    i_vocab_try = dict()

    # Choose language
    i_vocab_try = choose_a_language(i_vocab_try, vocab)
    if i_vocab_try["input_language"] == -1:
        break

    # Pick a random word
    i_vocab_try = choose_a_word(i_vocab_try, vocab)

    # Take a guess
    i_vocab_try = prompt_question(
        i_vocab_try,
        print_info={
            "nb_known_words_session": nb_known_words_session,
            "tries_session": tries_session,
        },
    )

    # Check answer
    i_vocab_try = check_answer(i_vocab_try, all_vocab)

    # Do we want to stop?
    if i_vocab_try["your_guess"] == "STOP IT":
        print(
            "Your last word was '{}={}'".format(
                i_vocab_try["word_input_language"],
                i_vocab_try["word_output_language"],
            )
        )
        break

    # If correct answer
    if i_vocab_try["is_it_correct"]:
        i_vocab_try = if_correct(i_vocab_try)
        nb_known_words_session += 1

    # If wrong answer
    if not i_vocab_try["is_it_correct"]:
        i_vocab_try = if_not_correct(i_vocab_try, vocab, all_vocab)

    # Add to historical data
    historical_data = add_historical_data(historical_data, i_vocab_try)

    # Update vocab
    vocab = update_vocab(vocab, i_vocab_try)

    tries_session += 1

In [ ]:
# Update and save historical data
update_historical_data(historical_data, test=is_it_a_test, test_name=test_name)

In [ ]:
# Update and save vocab
finalize_vocab(vocab, test=is_it_a_test, test_name=test_name)

In [ ]:
# prepare next session

In [ ]:
! python -m luigi --module src.luigi_pipeline PrepareNextSession --local-scheduler

In [ ]:
# info on words

In [ ]:
info_vocab(test=is_it_a_test, test_name=test_name)